# Step 1: Install Required Libraries

In [15]:
!pip install kaggle
!pip install datasets
!pip install evaluate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl

# Step 2: Import and Set Up

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [16]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [17]:
!kaggle datasets download azouzmaroua/algeria-ultrasound-images-thyroid-dataset-auitd --force
!unzip algeria-ultrasound-images-thyroid-dataset-auitd.zip

Dataset URL: https://www.kaggle.com/datasets/azouzmaroua/algeria-ultrasound-images-thyroid-dataset-auitd
License(s): unknown
100% 103M/103M [00:05<00:00, 25.0MB/s]
100% 103M/103M [00:05<00:00, 19.5MB/s]
Archive:  algeria-ultrasound-images-thyroid-dataset-auitd.zip
  inflating: dataset thyroid/test/Benign/2_0.jpg  
  inflating: dataset thyroid/test/Benign/2_1.jpg  
  inflating: dataset thyroid/test/Benign/2_10.jpg  
  inflating: dataset thyroid/test/Benign/2_11.jpg  
  inflating: dataset thyroid/test/Benign/2_12.jpg  
  inflating: dataset thyroid/test/Benign/2_13.jpg  
  inflating: dataset thyroid/test/Benign/2_14.jpg  
  inflating: dataset thyroid/test/Benign/2_15.jpg  
  inflating: dataset thyroid/test/Benign/2_16.jpg  
  inflating: dataset thyroid/test/Benign/2_17.jpg  
  inflating: dataset thyroid/test/Benign/2_18.jpg  
  inflating: dataset thyroid/test/Benign/2_19.jpg  
  inflating: dataset thyroid/test/Benign/2_2.jpg  
  inflating: dataset thyroid/test/Benign/2_20.jpg  
  inflatin

# Prepare data for training
From local file to pandas dataframe

In [18]:
import kaggle
import os
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split

seed_num = 112233

# โหลดชุดข้อมูล
data_root_path = "/content/dataset thyroid/"
data_split = os.listdir(data_root_path)

df_total = pd.DataFrame()
for split in data_split:
  data_path = os.path.join(data_root_path, split)
  class_list = os.listdir(data_path)

  for class_name in class_list:
    data_temp_pd = pd.DataFrame()
    file_path = os.path.join(data_path, class_name)
    file_list = os.listdir(file_path)
    file_list = [file_name for file_name in file_list if file_name.endswith(".jpg")]
    full_file_path = [os.path.join(file_path,file_name) for file_name in file_list]

    data_temp_pd['id'] = file_list
    data_temp_pd['image'] = full_file_path
    data_temp_pd['label'] = class_name

    df_total = pd.concat([df_total, data_temp_pd])

df_total['label'] = df_total['label'].str.lower()

In [ ]:
df_total

,id,image,label
0,tiroides (31).jpg,/content/dataset thyroid/test/normal thyroid/t...,normal thyroid
1,tiroides (34).jpg,/content/dataset thyroid/test/normal thyroid/t...,normal thyroid
2,tiroides (37).jpg,/content/dataset thyroid/test/normal thyroid/t...,normal thyroid
3,tiroides (35).jpg,/content/dataset thyroid/test/normal thyroid/t...,normal thyroid
4,tiroides2.jpg,/content/dataset thyroid/test/normal thyroid/t...,normal thyroid
...,...,...,...
287,5_24.jpg,/content/dataset thyroid/train/Malignant/5_24.jpg,malignant
288,4B_32.jpg,/content/dataset thyroid/train/Malignant/4B_32...,malignant
289,5_23.jpg,/content/dataset thyroid/train/Malignant/5_23.jpg,malignant
290,5_12.jpg,/content/dataset thyroid/train/Malignant/5_12.jpg,malignant


In [26]:
id2label = df_total[['label']].drop_duplicates().reset_index(drop = True)['label'].to_dict()
label2id  = {key:value for value, key in id2label.items()}
df_total['label'] = df_total['label'].map(label2id)

df_train, df_test = train_test_split(df_total, test_size=0.05, random_state=seed_num, stratify= df_total['label'])

In [ ]:
id2label

{0: 'benign', 1: 'normal thyroid', 2: 'malignant'}

In [ ]:
label2id

{'benign': 0, 'normal thyroid': 1, 'malignant': 2}

In [27]:
# Change from pandas dataframe to datasets
from datasets import load_dataset, Image
from datasets import Dataset

dataset_train = Dataset.from_dict(df_train)
dataset_train = dataset_train.cast_column("image", Image())

dataset_test = Dataset.from_dict(df_test)
dataset_test = dataset_test.cast_column("image", Image())

# Model 1: ViT

In [28]:
from transformers import AutoImageProcessor, DefaultDataCollator
from torchvision.transforms import CenterCrop, Compose, Normalize, RandomResizedCrop, Resize, ToTensor, Lambda, RandomHorizontalFlip
import io

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None

elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")

train_transforms = Compose([RandomResizedCrop(size), RandomHorizontalFlip(), ToTensor(), normalize])
val_transforms = Compose([Resize(size), CenterCrop(crop_size),ToTensor(), normalize])

def transforms_train(examples):
    examples["pixel_values"] = [train_transforms(img.convert("RGB")) for img in examples["image"]]
    return examples

def transforms_validate(examples):
    examples["pixel_values"] = [val_transforms(img.convert("RGB")) for img in examples["image"]]
    return examples

dataset_train = dataset_train.with_transform(transforms_train)
dataset_test = dataset_test.with_transform(transforms_validate)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

In [ ]:
import evaluate
import numpy as np
import torch

# Evaluation function

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
# Training loop
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(label2id.keys()),
    id2label=id2label,
    label2id=label2id,
)

training_args = TrainingArguments(
    output_dir = "./",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,            # Main value for fine tuning
    per_device_train_batch_size=50,# Main value for fine tuning
    gradient_accumulation_steps=1, # Main value for fine tuning
    per_device_eval_batch_size=50, # Main value for fine tuning
    num_train_epochs=30,           # Main value for fine tuning, starting from low number !!
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

train_results = trainer.train()

# Metric F1 Score

In [29]:
import evaluate
import numpy as np
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, predictions)

   # Calculate precision, recall, and F1-score
    precision = precision_score(labels, predictions, average='weighted',zero_division=0)
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [31]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(label2id.keys()),
    id2label=id2label,
    label2id=label2id,
)

training_args = TrainingArguments(
    output_dir = "./",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate= 3e-5,            # Main value for fine tuning
    per_device_train_batch_size=30,# Main value for fine tuning
    gradient_accumulation_steps=2, # Main value for fine tuning
    per_device_eval_batch_size=50, # Main value for fine tuning
    num_train_epochs=30,           # Main value for fine tuning, starting from low number !!
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=True,
    hub_model_id="agent593/Thyroid-Ultrasound-Image-Classification-ViTModel"
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
train_results = trainer.train()
results = trainer.evaluate()
print(results)

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.055400,1.014120,0.517857,0.273052,0.517857,0.357568
2,0.874800,0.822580,0.535714,0.433766,0.535714,0.389711
3,0.666900,0.617666,0.678571,0.480243,0.678571,0.555921
4,0.563400,0.515189,0.803571,0.857589,0.803571,0.776014
5,0.503500,0.446960,0.821429,0.828061,0.821429,0.811942
6,0.477200,0.398235,0.875000,0.883634,0.875000,0.876399
7,0.457000,0.417487,0.857143,0.883734,0.857143,0.859105
8,0.439600,0.359924,0.875000,0.894572,0.875000,0.876726


In [ ]:
import pandas as pd
pd.DataFrame(trainer.state.log_history)

In [ ]:
pd.DataFrame(trainer.state.log_history).head(21)

,loss,grad_norm,learning_rate,epoch,step,eval_loss,eval_accuracy,eval_precision,eval_recall,eval_f1,eval_runtime,eval_samples_per_second,eval_steps_per_second,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss
0,1.0668,0.997251,0.000006,0.277778,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0280,1.030907,0.000013,0.555556,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.9971,0.717427,0.000019,0.833333,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,1.000000,18,0.900731,0.517857,0.268176,0.517857,0.353361,0.8492,65.948,2.355,NaN,NaN,NaN,NaN,NaN
4,0.8748,0.917210,0.000026,1.111111,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.8421,0.860684,0.000032,1.388889,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.7754,1.111447,0.000039,1.666667,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.7295,0.987517,0.000045,1.944444,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,2.000000,36,0.706379,0.660714,0.473586,0.660714,0.541335,0.8657,64.686,2.310,NaN,NaN,NaN,NaN,NaN
9,0.6422,0.877137,0.000052,2.222222,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
trainer.save_model("/content")

In [35]:
from transformers import pipeline
classifier = pipeline("image-classification", model="agent593/Thyroid-Ultrasound-Image-Classification-ViTModel")

config.json:   0%|          | 0.00/785 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

In [36]:
image = "/content/dataset thyroid/test/Malignant/4A_0.jpg"
classifier(image)

[{'label': 'malignant', 'score': 0.9519590735435486},
 {'label': 'benign', 'score': 0.031021935865283012},
 {'label': 'normal thyroid', 'score': 0.01701892353594303}]